In [ ]:
!pwd

In [ ]:
!python --version

In [ ]:
import tensorflow as tf
import numpy as np
import sys

In [ ]:
from magenta.models.rl_tuner import rl_tuner
from magenta.models.rl_tuner import rl_tuner_ops

In [ ]:
#设定输出文件存储路径
SAVE_PATH = "/tmp/rl_tuner/2020.3.7-03"

In [ ]:
# Model parameter settings
ALGORITHM = 'q'
REWARD_SCALER = 1
OUTPUT_EVERY_NTH = 50000
NUM_NOTES_IN_COMPOSITION = 64
PRIME_WITH_MIDI = False
#NOTE_RNN_CHECKPOINT_DIR = '/root/magenta/magenta/models/rl_tuner/checkpoint'
#NOTE_RNN_CHECKPOINT_FILE = '/root/magenta/magenta/models/rl_tuner/checkpoint/note_rnn.ckpt'

In [ ]:
rl_tuner_hparams = tf.contrib.training.HParams(random_action_probability=0.1,
                                               store_every_nth=1,
                                               train_every_nth=5,
                                               minibatch_size=32,
                                               discount_rate=0.5,
                                               max_experience=100000,
                                               target_network_update_rate=0.01)

# Train network

In [ ]:
rl_net = rl_tuner.RLTuner(SAVE_PATH, 
                          dqn_hparams=rl_tuner_hparams, 
                          algorithm=ALGORITHM,
                          reward_scaler=REWARD_SCALER,
                          output_every_nth=OUTPUT_EVERY_NTH,
                          num_notes_in_melody=NUM_NOTES_IN_COMPOSITION)

In [ ]:
# Generate initial music sequence before training with RL
rl_net.generate_music_sequence(visualize_probs=True, title='pre_rl', length=32)

In [ ]:
rl_net.train(num_steps=1000000, exploration_period=500000)

In [ ]:
# Plot the rewards received during training. Improves as chance of random exploration action decreases.
rl_net.plot_rewards()

In [ ]:
# Plot rewards received during calls to evaluation function throughout training. 
# Does not include exploration or random actions.
rl_net.plot_evaluation()